In [360]:
import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plot
%matplotlib inline

In [361]:
#Read text file created from SAP
f=open("yva1.txt","r")

In [362]:
#Read Excel file 
exl=pd.read_excel('Book1.xlsx', converters={'item#':str,'part#':str, 'deliverydate':pd.to_datetime})
#https://stackoverflow.com/questions/32591466/python-pandas-how-to-specify-data-types-when-reading-an-excel-file

In [363]:
lines= f.readlines()

In [364]:
parse_flag=False
total=[]

for i in lines:
    if i[0]=='-':
        parse_flag=True
        continue
    if parse_flag==True and i[0]=='|':
        
        line=i.split()
        item=line[1];  so=line[2];
        newdate=datetime.datetime.strptime(line[3],  "%Y-%m-%d")
       
        delta=(newdate-datetime.datetime.now()).days
        
        single=[item, so, newdate, delta]
        total.append(single)
        parse_flag=False
    
df=pd.DataFrame(total)

In [365]:
#ID created for comparision and merge
df['id']=df[1] + df[0]
exl['id']=exl['part#'] + exl['item#']

In [366]:
kk=df.merge(exl)

In [369]:
kk['delta']=kk['deliverydate'] - kk[2]

In [373]:
#To save excel file 
writer=pd.ExcelWriter('output.xlsx')
kk.to_excel(writer,'Sheet1')
writer.save()